<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_nn_cde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
def sliding_window_transformation(dataset, window_size=10):
  '''
  A function to transform time-series data into tabular by sliding window
  Args:
    dataset: the target time-series data.
    window_size: default size=10
  Returns: the tansformed tabular data
  '''
  dataX = []
  for i in range(len(dataset)-window_size+1):
      a = dataset[i:(i+window_size)]
      dataX.append(a)
  return np.array(dataX)

In [0]:
#uploaded = files.upload()
'''Load data'''
#non_time_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))[['unix_ts']]
non_time_data = pd.read_csv('Electricity_WHE.csv')['unix_ts']

In [7]:
uploaded = files.upload()
CDE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CDE.csv']))['P']
#CDE_data = pd.read_csv('Electricity_CDE.csv')['P']

Saving Electricity_CDE.csv to Electricity_CDE.csv


In [9]:
#uploaded = files.upload()
#nn_data = pd.read_csv(io.BytesIO(uploaded['nn_data.csv']))
nn_data = pd.read_csv('nn_data.csv')
print(nn_data.shape)

(1048141, 4)


In [0]:
scaler = MinMaxScaler()
nn_data = scaler.fit_transform(nn_data.astype(float))

In [0]:
window_size=60
non_time_data = pd.to_datetime(non_time_data,unit='s')
whe_month = non_time_data.dt.month.as_matrix()[window_size-1:].reshape([-1,1])
whe_hour = non_time_data.dt.hour.as_matrix()[window_size-1:].reshape([-1,1])
whe_weekday = non_time_data.dt.dayofweek.as_matrix()[window_size-1:].reshape([-1,1])
non_time = np.hstack([whe_month,whe_hour,whe_weekday])
non_time = np.concatenate([non_time[0:472500,:],non_time[475500:,:]],axis=0)
data_arr = np.hstack([nn_data,non_time])

dataX = pd.get_dummies(pd.DataFrame(data_arr),columns=[4,5,6]).values
cde_Y = CDE_data.as_matrix()[window_size-1:].reshape([-1,1])
cde_Y = np.concatenate([cde_Y[0:472500,:],cde_Y[475500:,:]],axis=0)
categorized_cde_Y = np.ones(cde_Y.shape)*3
categorized_cde_Y[[np.where(cde_Y==0)[0]],:]=0
categorized_cde_Y[[np.where((cde_Y>0)&(cde_Y<=350))[0]],:]=1
categorized_cde_Y[[np.where((cde_Y>4400)&(cde_Y<=5300))[0]],:]=2
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cdeY_1hot = encoder.fit_transform(categorized_cde_Y)

In [22]:
max_indice = np.argmax(cdeY_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    1026791
2      11216
1      10095
3         39
dtype: int64

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dataX, cdeY_1hot, test_size=0.1, shuffle=True)

In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:9000,1:9000,2:9000,3:0})
X_train, y_train = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X_train,y_train],axis=1)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.0001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=1024
unit_num=128

In [0]:
#define the Feedforward Model
'''Feedforward Model Definition'''
tf.reset_default_graph()

''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, 47],name='ff_x')
tf_y = tf.placeholder(tf.int32, [None, 4],name='ff_y')

l1 = tf.layers.dense(tf_x,64,activation=tf.nn.leaky_relu,name='ff_l1')
l2 = tf.layers.dense(l1,128,activation=tf.nn.leaky_relu,name='ff_l2')
l3 = tf.layers.dense(l2,256,activation=tf.nn.leaky_relu,name='ff_l3')
l4 = tf.layers.dense(l3,128,activation=tf.nn.leaky_relu,name='ff_l4')
l5 = tf.layers.dense(l4,64,activation=tf.nn.leaky_relu,name='ff_l5')
l6 = tf.layers.dense(l5,32,activation=tf.nn.leaky_relu,name='ff_l6')
l7 = tf.layers.dense(l6,16,activation=tf.nn.leaky_relu,name='ff_l7')
l8 = tf.layers.dense(l7,8,activation=tf.nn.leaky_relu,name='ff_l8')
pred = tf.layers.dense(l8,4,activation=tf.nn.relu,name='ff_pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


In [0]:
sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net/save_net_rnn.ckpt')

flag = False
for j in range(0,100):
    print('###iteration: '+str(j)+'###')
    batch_index = np.random.choice(27000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-4].reshape([-1,47])
    batch_y = batch_train[:,-4:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],47]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if acc_train>=0.99:
      print(j)
      flag = True
      break
    if(flag==True):
        print(flag)
        break
    if(j//100==0):
      save_path = saver.save(sess, "my_net/save_net_rnn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],47]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()


INFO:tensorflow:Restoring parameters from my_net/save_net_rnn.ckpt
###iteration: 0###
train loss= 0.0966, Acc=0.97
Test Acc=0.93
[[341  11   0   0]
 [ 22 320   0   0]
 [  0   0 330   0]
 [  0   0   0   0]]
###iteration: 1###
train loss= 0.1165, Acc=0.96
Test Acc=0.93
[[324  13   1   0]
 [ 24 311   2   0]
 [  0   1 348   0]
 [  0   0   0   0]]
###iteration: 2###
train loss= 0.0976, Acc=0.97
Test Acc=0.93
[[324  10   0   0]
 [ 23 312   0   0]
 [  0   1 354   0]
 [  0   0   0   0]]
###iteration: 3###
train loss= 0.1095, Acc=0.96
Test Acc=0.93
[[324  15   1   0]
 [ 21 321   1   0]
 [  0   0 341   0]
 [  0   0   0   0]]
###iteration: 4###
train loss= 0.1042, Acc=0.96
Test Acc=0.94
[[332  10   0   0]
 [ 27 293   0   0]
 [  0   0 362   0]
 [  0   0   0   0]]
###iteration: 5###
train loss= 0.1065, Acc=0.97
Test Acc=0.94
[[326  12   0   0]
 [ 17 313   5   0]
 [  0   0 351   0]
 [  0   0   0   0]]
###iteration: 6###
train loss= 0.1072, Acc=0.96
Test Acc=0.93
[[304  17   0   0]
 [ 24 331   1   0]